In [1]:
### https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
rd = pd.read_csv(r'C:\Users\chpar10\OneDrive - IKEA\DA\Call_trend.csv')
rd = rd[:-1]
rd = rd.iloc[:,:-16]

In [3]:
rd.head()

,date,offered,week,t1,t2,t3,trend,sun,mon,tue,...,wk51,wk52,wk53,holiday,holiday-1,holiday-2,holiday-3,GM,GY,ONLINE
0,2015-01-01,688.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1.0,0.0,0.0
1,2015-01-02,1096.0,1.0,2.0,4.0,64.0,70.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,1.0,0.0,0.0
2,2015-01-03,980.0,1.0,3.0,9.0,729.0,741.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0
3,2015-01-04,546.0,2.0,4.0,16.0,4096.0,4116.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0
4,2015-01-05,1074.0,2.0,5.0,25.0,15625.0,15655.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0


In [4]:
rd = rd.drop(['t1', 't2', 't3'], axis=1)

In [5]:
idx = [not i for i in rd.offered.isna()]
dataset = rd[idx]
validset = rd.iloc[-7:,2:]

In [6]:
dataset

,date,offered,week,trend,sun,mon,tue,wed,thu,fri,...,wk51,wk52,wk53,holiday,holiday-1,holiday-2,holiday-3,GM,GY,ONLINE
0,2015-01-01,688.0,1.0,3.000000e+00,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1.0,0.0,0.0
1,2015-01-02,1096.0,1.0,7.000000e+01,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,1.0,0.0,0.0
2,2015-01-03,980.0,1.0,7.410000e+02,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0
3,2015-01-04,546.0,2.0,4.116000e+03,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0
4,2015-01-05,1074.0,2.0,1.565500e+04,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
5,2015-01-06,816.0,2.0,4.669800e+04,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
6,2015-01-07,823.0,2.0,1.177050e+05,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
7,2015-01-08,811.0,2.0,2.622160e+05,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
8,2015-01-09,742.0,2.0,5.315310e+05,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
9,2015-01-10,684.0,2.0,1.000110e+06,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0


In [7]:
X, y = dataset.iloc[:,2:], dataset.offered

In [8]:
xgb.DMatrix(X, y)

c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4, random_state=42)
print(len(y_train), len(y_test))

999 666


In [10]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dvalid = xgb.DMatrix(validset)

In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np
# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))


rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_predictions))
print("Baseline RMSE is {:.2f}".format(mae_baseline))

Baseline MAE is 127.63
Baseline RMSE is 127.63


In [12]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
    # Eval with Mean Absolute Error
    #'eval_metric': "rmse",
    'eval_metric': "mae",
}

In [13]:
num_boost_round = 999

In [14]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-mae:451.68
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:319.503
[2]	Test-mae:226.903
[3]	Test-mae:162.42
[4]	Test-mae:119.048
[5]	Test-mae:91.3742
[6]	Test-mae:74.7237
[7]	Test-mae:65.3679
[8]	Test-mae:60.3342
[9]	Test-mae:57.2421
[10]	Test-mae:55.6311
[11]	Test-mae:54.7146
[12]	Test-mae:54.448
[13]	Test-mae:53.7669
[14]	Test-mae:53.6995
[15]	Test-mae:53.7316
[16]	Test-mae:53.5604
[17]	Test-mae:53.788
[18]	Test-mae:53.0881
[19]	Test-mae:52.8082
[20]	Test-mae:52.6144
[21]	Test-mae:52.6087
[22]	Test-mae:52.5307
[23]	Test-mae:52.4036
[24]	Test-mae:52.453
[25]	Test-mae:52.1154
[26]	Test-mae:52.2168
[27]	Test-mae:52.3045
[28]	Test-mae:52.4973
[29]	Test-mae:52.3533
[30]	Test-mae:52.0596
[31]	Test-mae:52.2374
[32]	Test-mae:52.3829
[33]	Test-mae:52.3322
[34]	Test-mae:52.2646
[35]	Test-mae:52.3551
[36]	Test-mae:52.1674
[37]	Test-mae:52.1754
[38]	Test-mae:52.1514
[39]	Test-mae:52.1187
[40]	Test-mae:52.0946
Stopping. Best iteration:
[30]	Test-mae:52.0596



In [15]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
#    metrics={'rmse'},
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,454.067444,1.122351,453.424506,7.898343
1,320.686615,1.029687,320.399200,6.824642
2,226.774695,0.664001,226.327908,6.262598
3,161.312454,0.502560,162.369394,6.415219
4,116.470389,0.404157,120.843651,5.002839
5,86.894046,0.914667,94.741182,5.289884
6,68.040172,0.772388,78.765396,4.958971
7,56.045631,0.662534,69.425672,4.650119
8,49.172763,0.837558,64.660730,4.676136
9,44.503174,1.351233,61.424168,4.328563


In [16]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    #for max_depth in range(9,12)
    #for min_child_weight in range(5,8)
    for max_depth in range(7,12)
    for min_child_weight in range(2,8)
]

In [17]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        #metrics={'rmse'},
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds".format(mean_mae, boost_rounds))
#    if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = (max_depth,min_child_weight)
#print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_mae))


    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].idxmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=7, min_child_weight=2
	MAE 55.866159800000005 for 25 rounds
CV with max_depth=7, min_child_weight=3
	MAE 55.45593720000001 for 29 rounds
CV with max_depth=7, min_child_weight=4
	MAE 55.473285 for 37 rounds
CV with max_depth=7, min_child_weight=5
	MAE 57.3535318 for 30 rounds
CV with max_depth=7, min_child_weight=6
	MAE 56.740166 for 39 rounds
CV with max_depth=7, min_child_weight=7
	MAE 55.2197914 for 39 rounds
CV with max_depth=8, min_child_weight=2
	MAE 55.7618896 for 33 rounds
CV with max_depth=8, min_child_weight=3
	MAE 55.8959466 for 22 rounds
CV with max_depth=8, min_child_weight=4
	MAE 55.5010634 for 23 rounds
CV with max_depth=8, min_child_weight=5
	MAE 56.4598998 for 32 rounds
CV with max_depth=8, min_child_weight=6
	MAE 53.9694216 for 37 rounds
CV with max_depth=8, min_child_weight=7
	MAE 55.806761800000004 for 29 rounds
CV with max_depth=9, min_child_weight=2
	MAE 56.3480638 for 34 rounds
CV with max_depth=9, min_child_weight=3
	MAE 55.7148232 for 22 rounds

In [18]:
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [19]:
gridsearch_params = [
    (subsample, colsample)
#    for subsample in [i/10. for i in range(7,11)]
#    for colsample in [i/10. for i in range(7,11)]
    for subsample in np.linspace(0.5, 1, 16)
    for colsample in np.linspace(0.5, 1, 16)
]

In [20]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds".format(mean_mae, boost_rounds))
#    if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = (subsample,colsample)
#print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_mae))

    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0


c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


	MAE 53.9694216 for 37 rounds
CV with subsample=1.0, colsample=0.9666666666666667
	MAE 55.6167826 for 36 rounds
CV with subsample=1.0, colsample=0.9333333333333333
	MAE 55.15521540000001 for 25 rounds
CV with subsample=1.0, colsample=0.9
	MAE 55.323324400000004 for 41 rounds
CV with subsample=1.0, colsample=0.8666666666666667
	MAE 55.5437986 for 25 rounds
CV with subsample=1.0, colsample=0.8333333333333333
	MAE 55.331442200000005 for 27 rounds
CV with subsample=1.0, colsample=0.8
	MAE 55.54684239999999 for 58 rounds
CV with subsample=1.0, colsample=0.7666666666666666
	MAE 56.639652999999996 for 22 rounds
CV with subsample=1.0, colsample=0.7333333333333334
	MAE 56.4451174 for 44 rounds
CV with subsample=1.0, colsample=0.7
	MAE 56.2023368 for 47 rounds
CV with subsample=1.0, colsample=0.6666666666666666
	MAE 55.7593842 for 26 rounds
CV with subsample=1.0, colsample=0.6333333333333333
	MAE 55.81902540000001 for 49 rounds
CV with subsample=1.0, colsample=0.6
	MAE 56.4694878 for 25 rounds
C

	MAE 56.460202800000005 for 19 rounds
CV with subsample=0.8333333333333333, colsample=0.6
	MAE 57.340500600000006 for 30 rounds
CV with subsample=0.8333333333333333, colsample=0.5666666666666667
	MAE 56.8778664 for 57 rounds
CV with subsample=0.8333333333333333, colsample=0.5333333333333333
	MAE 56.480945 for 40 rounds
CV with subsample=0.8333333333333333, colsample=0.5
	MAE 56.871163 for 47 rounds
CV with subsample=0.8, colsample=1.0
	MAE 56.1339004 for 19 rounds
CV with subsample=0.8, colsample=0.9666666666666667
	MAE 56.261449400000004 for 27 rounds
CV with subsample=0.8, colsample=0.9333333333333333
	MAE 56.79076719999999 for 17 rounds
CV with subsample=0.8, colsample=0.9
	MAE 56.41429519999999 for 40 rounds
CV with subsample=0.8, colsample=0.8666666666666667
	MAE 55.7896286 for 36 rounds
CV with subsample=0.8, colsample=0.8333333333333333
	MAE 56.34568420000001 for 37 rounds
CV with subsample=0.8, colsample=0.8
	MAE 56.2788856 for 37 rounds
CV with subsample=0.8, colsample=0.76666

	MAE 55.819856400000006 for 19 rounds
CV with subsample=0.6333333333333333, colsample=0.7
	MAE 57.741436 for 19 rounds
CV with subsample=0.6333333333333333, colsample=0.6666666666666666
	MAE 56.8646086 for 48 rounds
CV with subsample=0.6333333333333333, colsample=0.6333333333333333
	MAE 56.5895896 for 29 rounds
CV with subsample=0.6333333333333333, colsample=0.6
	MAE 56.4760666 for 28 rounds
CV with subsample=0.6333333333333333, colsample=0.5666666666666667
	MAE 57.281385799999995 for 50 rounds
CV with subsample=0.6333333333333333, colsample=0.5333333333333333
	MAE 59.3154762 for 20 rounds
CV with subsample=0.6333333333333333, colsample=0.5
	MAE 57.1413644 for 50 rounds
CV with subsample=0.6, colsample=1.0
	MAE 56.72948 for 50 rounds
CV with subsample=0.6, colsample=0.9666666666666667
	MAE 55.742877 for 68 rounds
CV with subsample=0.6, colsample=0.9333333333333333
	MAE 55.590500799999994 for 27 rounds
CV with subsample=0.6, colsample=0.9
	MAE 56.8596374 for 37 rounds
CV with subsample=

In [21]:
params['subsample'] = best_params[0] #0.7
params['colsample_bytree'] = best_params[1] #0.7

In [22]:
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .075, .05, .025, .01, .0075, .005, .0025, .001]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
   
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10)
    
    # Update best score
#    mean_mae = cv_results['test-rmse-mean'].min()
#    boost_rounds = cv_results['test-rmse-mean'].idxmin()
#    print("\tRMSE {} for {} rounds\n".format(mean_mae, boost_rounds))
#   if mean_mae < min_mae:
#        min_mae = mean_mae
#        best_params = eta
#print("Best params: {}, RMSE: {}".format(best_params, min_mae))

    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].idxmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CV with eta=0.3
	MAE 53.9694216 for 37 rounds

CV with eta=0.2
	MAE 55.621895599999995 for 43 rounds

CV with eta=0.1
	MAE 55.3743056 for 85 rounds

CV with eta=0.075
	MAE 55.667219599999996 for 97 rounds

CV with eta=0.05
	MAE 55.1820434 for 191 rounds

CV with eta=0.025
	MAE 55.847448400000005 for 250 rounds

CV with eta=0.01
	MAE 55.803679599999995 for 549 rounds

CV with eta=0.0075
	MAE 55.576549 for 997 rounds

CV with eta=0.005
	MAE 56.0973434 for 998 rounds

CV with eta=0.0025
	MAE 75.814844 for 998 rounds

CV with eta=0.001
	MAE 242.146231 for 998 rounds

Best params: 0.3, MAE: 53.9694216


In [23]:
params['eta'] = best_params #.1

In [24]:
params

{'max_depth': 8,
 'min_child_weight': 6,
 'eta': 0.3,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'mae'}

In [25]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-mae:451.68
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:319.002
[2]	Test-mae:227.021
[3]	Test-mae:164.299
[4]	Test-mae:120.758
[5]	Test-mae:92.3152
[6]	Test-mae:75.5337
[7]	Test-mae:65.273
[8]	Test-mae:59.9236
[9]	Test-mae:57.1273
[10]	Test-mae:55.4863
[11]	Test-mae:54.5093
[12]	Test-mae:53.3221
[13]	Test-mae:53.1258
[14]	Test-mae:52.9651
[15]	Test-mae:52.8147
[16]	Test-mae:52.3032
[17]	Test-mae:52.103
[18]	Test-mae:52.0453
[19]	Test-mae:51.9548
[20]	Test-mae:51.9007
[21]	Test-mae:51.8514
[22]	Test-mae:52.0138
[23]	Test-mae:52.2052
[24]	Test-mae:52.2613
[25]	Test-mae:52.3336
[26]	Test-mae:52.468
[27]	Test-mae:52.4183
[28]	Test-mae:52.4492
[29]	Test-mae:52.4674
[30]	Test-mae:52.3362
[31]	Test-mae:52.3169
Stopping. Best iteration:
[21]	Test-mae:51.8514



In [26]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-mae:451.68
[1]	Test-mae:319.002
[2]	Test-mae:227.021
[3]	Test-mae:164.299
[4]	Test-mae:120.758
[5]	Test-mae:92.3152
[6]	Test-mae:75.5337
[7]	Test-mae:65.273
[8]	Test-mae:59.9236
[9]	Test-mae:57.1273
[10]	Test-mae:55.4863
[11]	Test-mae:54.5093
[12]	Test-mae:53.3221
[13]	Test-mae:53.1258
[14]	Test-mae:52.9651
[15]	Test-mae:52.8147
[16]	Test-mae:52.3032
[17]	Test-mae:52.103
[18]	Test-mae:52.0453
[19]	Test-mae:51.9548
[20]	Test-mae:51.9007
[21]	Test-mae:51.8514


In [27]:
mean_absolute_error(best_model.predict(dtest), y_test)

51.85138590128214

In [28]:
pd.DataFrame([best_model.predict(dtest), np.array(y_test)]).T

,0,1
0,609.030823,617.0
1,551.635315,538.0
2,661.818054,635.0
3,519.661011,477.0
4,549.061401,540.0
5,475.379211,504.0
6,599.513855,513.0
7,601.821228,721.0
8,526.746887,506.0
9,542.348694,508.0


In [29]:
best_model.predict(dvalid)

array([506.3674 , 847.673  , 598.2584 , 574.787  , 574.55896, 568.25665,
       499.97275], dtype=float32)

In [30]:
best_model.predict(dvalid).sum()

4169.8745

In [31]:
import matplotlib.pyplot as plt
xgb.plot_importance(best_model)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()

<Figure size 640x480 with 1 Axes>